## Predictive modeling file
This file contains code for all the predictive modeling tasks performed for the MSc thesis.

### Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import groupby
#!conda remove scipy scikit-learn -y
#!conda install scipy scikit-learn -y
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_error
from sklearn.metrics import r2_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, hamming_loss, f1_score
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict, cross_val_score, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import precision_recall_curve, average_precision_score
#%pip install mord
from mord import LogisticAT
from sklearn.metrics import classification_report, roc_auc_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.svm import SVC
#%pip install panelsplit
#from panelsplit import PanelSplit
from sklearn.model_selection import BaseCrossValidator
from sklearn.utils.validation import indexable
from itertools import chain
from statsmodels.miscmodels.ordinal_model import OrderedModel
from sklearn.base import BaseEstimator, RegressorMixin
from pandas.plotting import autocorrelation_plot
#%pip install imblearn
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingRegressor
#%pip install xgboost
import xgboost as xgb
from sklearn.utils.class_weight import compute_sample_weight
from scipy.stats import boxcox
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTENC
#%pip install smogn
from smogn import smoter
from sklearn.metrics import f1_score, roc_auc_score, classification_report, RocCurveDisplay
#%pip install lifelines
from lifelines import CoxPHFitter
from lifelines.utils import k_fold_cross_validation
from lifelines import CoxPHFitter
from lifelines.statistics import proportional_hazard_test
#%pip install scikit-survival
from sklearn.metrics import f1_score, roc_auc_score, classification_report, precision_recall_curve, auc
from sklearn.preprocessing import QuantileTransformer
import pickle


### Data load

In [ ]:
import os
os.chdir('C:/Users/Nik/Documents/thesis/PEP data copy/Original Stata files/')
path = "./PEP_all data_long_20220316.dta"
path2 = "./PEP_demographics_20220316.dta"
df = pd.read_stata(path)
df2 = pd.read_stata(path2)
df2 = df2.rename(columns={'StudyID': 'studyID'})
df_merged = pd.merge(df, df2, on='studyID', how='left')
df = df_merged


### Data pre-processing

In [ ]:
# defining all useful columns
disease_columns = ['heartattack', 'strokes', 'congestiveHD', 'carcinoma', 'hipfracture', 'chronicLung', 'Hypertension', 'arthritis', 'DM_TG']
outcome_columns = ['N_adl4dis', 'N_IADL5', 'N_mob4dis']
prob_b_columns = [col for col in df.columns if col.startswith('prob_b')]
other_columns = ['SCESD_ge16', 'BMI_ge30']
confounder_columns = ['BMI_3cp', 'age_fu', 'female', 'white', 'vis3cat', 'hear3cat', 'smoker_fu', 'mu3', 'rx_fu', 'hospstay']

In [ ]:
def impute_disease(df, disease_columns, date_col):
    """
    Impute NaN values in disease columns based on historical data for each studyID.
    If a disease was ever reported (1) by an individual, subsequent NaNs are set to 1.
    If a disease was never reported before a NaN, it is set to 0.

    Args:
    df (DataFrame): The dataframe containing the data.
    disease_columns (list): List of columns that are disease indicators.
    date_col (str): Column name for the date to ensure chronological order.

    Returns:
    DataFrame: The DataFrame with NaNs in disease columns imputed based on historical data.
    """
    # Sort the df chronologically by studyID and date
    df = df.sort_values(by=['studyID', date_col])

    # Process each participant separately
    for studyID, group in df.groupby('studyID'):
        # Iterate through each disease column
        for disease in disease_columns:
            # Find the first instance where the disease was reported
            first_report_index = group[disease].first_valid_index()

            # Create to identify rows before the first report
            if first_report_index is not None:
                before_first_report = group.index < first_report_index
            else:
                before_first_report = pd.Series(False, index=group.index)

            # Apply cumulative max to forward-fill reported diseases; NaNs before the first report are unaffected
            group[disease] = group[disease].cummax()

            # Fill NaNs before the first reported instance with 0 (since disease was never reported)
            group.loc[before_first_report, disease] = group.loc[before_first_report, disease].fillna(0)

            # Backfill any remaining NaNs after the first report with 1 (assuming disease persists)
            group[disease] = group[disease].fillna(1)

        # Place the modified group back into the main DataFrame
        df.loc[group.index, disease_columns] = group[disease_columns]

    return df

# example
df = impute_disease(df, disease_columns, 'intdate')

# Check the results
#print(df.head())


In [ ]:
#prob_b_columns = [col for col in df.columns if col.startswith('prob_b')]
df[prob_b_columns] = df[prob_b_columns].fillna(0)
#df[disease_columns] = df[disease_columns].fillna(0)
df[other_columns] = df[other_columns].fillna(0)
df[outcome_columns] = df[outcome_columns].fillna(0)

 # here i impute tiny bits of data that were missing from symptom columns, i fill 0 bc
# i assume that the symptoms were simply not reported

df[prob_b_columns] = df[prob_b_columns].astype(int)
df[disease_columns] = df[disease_columns].astype(int)
df[other_columns] = df[other_columns].astype(int)
df[outcome_columns] = df[outcome_columns].astype(int)


In [ ]:

# Ensure intdate is in datetime format
df['intdate'] = pd.to_datetime(df['intdate'])

# Sort by studyID and interview date
df = df.sort_values(by=['studyID', 'intdate'])



#### Map column names
# List of detailed symptoms based on PEP study
detailed_symptoms = [
    "pain or stiffness in your joints",
    "pain or stiffness in your back",
    "leg pain on walking",
    "weakness of your arms or legs",
    "swelling in your feet or ankles",
    "been fatigued (no energy/very tired)",
    "difficulty breathing or shortness of breath",
    "chest pain or tightness",
    "poor or decreased vision",
    "been dizzy or unsteady on your feet",
    "a fall or injury",
    "been afraid of falling",
    "cold or flu symptoms",
    "difficulty with sleeping",
    "nausea, vomiting, diarrhea, or other stomach (abdominal) problem",
    "a problem with your memory or difficulty thinking",
    "been depressed",
    "been anxious or worried",
    "frequent or painful urination",
    "lost control of your urine and wet yourself",
    "has a family member or friend become seriously ill or had an accident",
    "experienced the death or loss of a family member or friend",
    "a change in your medications",
    "a problem with alcohol"
]

# Starting column index for prob_b columns
start_index = 3

# Create a mapping of prob_b columns to the new symptom descriptions
column_mapping = {f'prob_b{i}': symptom for i, symptom in zip(range(start_index, start_index + len(detailed_symptoms)), detailed_symptoms)}

df.rename(columns=column_mapping, inplace=True)

prob_b_columns = list(column_mapping.values())

# Outcome columns
outcome_columns = ['N_adl4dis', 'N_IADL5', 'N_mob4dis']

# Initialize delta columns and combined_change
for col in outcome_columns:
    df['delta_' + col] = df[col] - df.groupby('studyID')[col].shift()
    df['change_' + col] = df['delta_' + col].apply(lambda x: 1 if x != 0 else 0)

df['combined_change'] = df[[f'change_{col}' for col in outcome_columns]].max(axis=1).astype(int)

# Calculate cumulative time since last report for each individual
df['cumulative_time'] = df.groupby('studyID')['TR'].cumsum()

# Initialize actual time until change columns
df['actual_time_until_change'] = np.nan

# Calculate actual time until the change occurred
for study_id, group in df.groupby('studyID'):
    change_indices = group.index[group['combined_change'] == 1].tolist()
    if change_indices:
        change_time = 0
        for idx in group.index:
            if idx in change_indices:
                df.loc[idx, 'actual_time_until_change'] = change_time
                change_time = 0
            change_time += group.loc[idx, 'TR']

# Calculate final intbloc value for each studyID
final_intbloc = df.groupby('studyID')['intbloc'].transform('max')

# Fill NaNs in actual_time_until_change for entries where no change occurred with the final intbloc value
df['actual_time_until_change'].fillna(final_intbloc, inplace=True)


# Filter participants with at least three observations
df = df.groupby('studyID').filter(lambda x: len(x) >= 2)

# Print the number of participants left after filtering
num_participants = df['studyID'].nunique()
print(f"Number of participants with at least two observations: {num_participants}")


# Calculate symptom frequency and duration per participant
for col in prob_b_columns:
    df[f'{col}_frequency'] = df.groupby('studyID')[col].cumsum()
    df[f'{col}_duration'] = df.groupby('studyID')[col].transform(lambda x: (x != 0).astype(int).groupby((x == 0).astype(int).cumsum()).cumsum())

# Calculate symptom onset for each symptom
for col in prob_b_columns:
    df[f'{col}_onset'] = df.groupby('studyID').apply(lambda x: (x[col] * x['TR']).cumsum() - (x[col] * x['TR']).cumsum().where(x[col] == 0).ffill().fillna(0)).values

# Calculate symptom recurrence for each symptom
for col in prob_b_columns:
    df[f'{col}_recurrence'] = df.groupby('studyID')[col].transform(lambda x: (x.diff().fillna(0) == 1).cumsum())

# Define the range for lagged features
lag_range = range(1, 2)

# Create lagged features for symptoms, TR, and temporal features
for col in prob_b_columns + ['TR'] + outcome_columns:
    for lag in lag_range:
        df[f'{col}_lag{lag}'] = df.groupby('studyID')[col].shift(lag)

# Create lagged features for additional temporal features
for col in [f'{symptom}_recurrence' for symptom in prob_b_columns] + \
           [f'{symptom}_onset' for symptom in prob_b_columns] + \
           [f'{symptom}_frequency' for symptom in prob_b_columns] + \
           [f'{symptom}_duration' for symptom in prob_b_columns]:
    for lag in lag_range:
        df[f'{col}_lag{lag}'] = df.groupby('studyID')[col].shift(lag)

# Drop rows with NaN values in any of the lagged feature columns
lagged_columns = [f'{col}_lag{lag}' for col in prob_b_columns + outcome_columns for lag in lag_range]
lagged_temporal_columns = [f'{col}_lag{lag}' for col in 
                           [f'{symptom}_recurrence' for symptom in prob_b_columns] +
                           [f'{symptom}_onset' for symptom in prob_b_columns] +
                           [f'{symptom}_frequency' for symptom in prob_b_columns] +
                           [f'{symptom}_duration' for symptom in prob_b_columns]
                           for lag in lag_range]
df = df.dropna(subset=lagged_columns + lagged_temporal_columns)

# Store the lagged feature names in a variable
lagged_symptom_features = [f'{col}_lag{lag}' for col in prob_b_columns for lag in lag_range]
lagged_outcomes = [f'{col}_lag{lag}' for col in outcome_columns for lag in lag_range]
lagged_tr_features = [f'TR_lag{lag}' for lag in lag_range]
lagged_temporal_features = [f'{symptom}_recurrence_lag{lag}' for symptom in prob_b_columns for lag in lag_range] + \
                           [f'{symptom}_onset_lag{lag}' for symptom in prob_b_columns for lag in lag_range] + \
                           [f'{symptom}_frequency_lag{lag}' for symptom in prob_b_columns for lag in lag_range] + \
                           [f'{symptom}_duration_lag{lag}' for symptom in prob_b_columns for lag in lag_range]

# Calculate total number of symptoms (lagged)
df['total_symptoms_lagged'] = df[lagged_symptom_features].sum(axis=1)

# Additional feature engineering: Calculate magnitude of changes
for col in outcome_columns:
    df[f'magnitude_change_{col}'] = df[f'delta_' + col].abs()

# Define feature sets
binary_features = prob_b_columns
numeric_features = [col for col in df.columns if col not in binary_features + ['actual_time_until_change', 'total_symptoms_lagged']]
temporal_features = [f'{col}_frequency' for col in prob_b_columns] + \
                    [f'{col}_duration' for col in prob_b_columns] + \
                    [f'{col}_onset' for col in prob_b_columns] + \
                    [f'{col}_recurrence' for col in prob_b_columns]

# Separate temporal features into lagged and current sets
current_temporal_features = temporal_features

# Final dataframe for modeling (do not dropna)
df = df.reset_index(drop=True)
# Check the structure of the dataframe
df.isna().sum()

# Plot the distribution of actual_time_until_change
#plt.figure(figsize=(10, 6))
#df['actual_time_until_change'].dropna().plot(kind='hist', bins=range(int(df['actual_time_until_change'].min()), int(df['actual_time_until_change'].max()) + 2), alpha=0.7, color='blue')
#plt.title('Distribution of Actual Time Until Change')
#plt.xlabel('Time Until Change (months)')
#plt.ylabel('Frequency')
#plt.xticks(range(int(df['actual_time_until_change'].min()), int(df['actual_time_until_change'].max()) + 1))
#plt.tight_layout()
#plt.show()


In [ ]:
# Calculate the end-of-life threshold (last 5 months)
df['end_of_life_threshold'] = df['ddate'] - pd.DateOffset(months=5)


# Remove the last 5 months of life for each participant who has died (Died == 1)
df_no_eol = df[~((df['Died'] == 1) & (df['intdate'] >= df['end_of_life_threshold']))]

df = df_no_eol

## Predictive modeling for TR

### Model 1.1 (baseline using only previous TR)

In [ ]:

# Extract features and target
X = df[lagged_tr_features]
y = df['TR']

# Separate the binary and numeric features
numeric_features = lagged_tr_features 

# Scale numeric features using QuantileTransformer
scaler = QuantileTransformer(output_distribution='normal')
X[numeric_features] = scaler.fit_transform(X[numeric_features])

# Train-test split
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

# Hyperparameter grid for RandomForestRegressor
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV for hyperparameter tuning
model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print(f"Best hyperparameters:\n{best_params}")

# Evaluate on training set
y_pred_train = best_model.predict(X_train)
train_mae = mean_absolute_error(y_train, y_pred_train)
train_r2 = r2_score(y_train, y_pred_train)
print(f"Train MAE: {train_mae}")
print(f"Train R-squared: {train_r2}")

# Evaluate on validation set
y_pred_val = best_model.predict(X_val)
val_mae = mean_absolute_error(y_val, y_pred_val)
val_r2 = r2_score(y_val, y_pred_val)
print(f"Validation MAE: {val_mae}")
print(f"Validation R-squared: {val_r2}")

# Evaluate on test set
y_pred_test = best_model.predict(X_test)
test_mae = mean_absolute_error(y_test, y_pred_test)
test_r2 = r2_score(y_test, y_pred_test)
print(f"Test MAE: {test_mae}")
print(f"Test R-squared: {test_r2}")

# Print best hyperparameters
print(f"Best hyperparameters:\n{best_params}")

# Plot feature importances for the best model
feature_importances = best_model.feature_importances_
features = X_train.columns

indices = np.argsort(feature_importances)[::-1]

plt.figure(figsize=(10, 6))
plt.title("Feature Importances")
plt.bar(range(X_train.shape[1]), feature_importances[indices], align="center")
plt.xticks(range(X_train.shape[1]), features[indices], rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()
plt.show()

# Save the model to a file 
with open('rfr_model_1_1.pkl', 'wb') as file:
    pickle.dump(best_model, file)


# Actual vs Predicted Plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_test, c='blue', marker='o', label='Test data')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linewidth=2)
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('Actual vs Predicted Plot')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()


### Model 1.2 (symptom-based model)

In [ ]:

# Extract features and target
X = df[lagged_tr_features + prob_b_columns + lagged_temporal_features]
y = df['TR']

# Separate the binary and numeric features
numeric_features = lagged_tr_features 

# Scale numeric features using QuantileTransformer
scaler = QuantileTransformer(output_distribution='normal')
X[numeric_features] = scaler.fit_transform(X[numeric_features])

# Train-test split
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

# Hyperparameter grid for RandomForestRegressor
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV for hyperparameter tuning
model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print(f"Best hyperparameters:\n{best_params}")

# Evaluate on training set
y_pred_train = best_model.predict(X_train)
train_mae = mean_absolute_error(y_train, y_pred_train)
train_r2 = r2_score(y_train, y_pred_train)
print(f"Train MAE: {train_mae}")
print(f"Train R-squared: {train_r2}")

# Evaluate on validation set
y_pred_val = best_model.predict(X_val)
val_mae = mean_absolute_error(y_val, y_pred_val)
val_r2 = r2_score(y_val, y_pred_val)
print(f"Validation MAE: {val_mae}")
print(f"Validation R-squared: {val_r2}")

# Evaluate on test set
y_pred_test = best_model.predict(X_test)
test_mae = mean_absolute_error(y_test, y_pred_test)
test_r2 = r2_score(y_test, y_pred_test)
print(f"Test MAE: {test_mae}")
print(f"Test R-squared: {test_r2}")

# Print best hyperparameters
print(f"Best hyperparameters:\n{best_params}")

# Plot feature importances for the best model
feature_importances = best_model.feature_importances_
features = X_train.columns

indices = np.argsort(feature_importances)[::-1]

plt.figure(figsize=(10, 6))
plt.title("Feature Importances")
plt.bar(range(X_train.shape[1]), feature_importances[indices], align="center")
plt.xticks(range(X_train.shape[1]), features[indices], rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()
plt.show()

# Save the model to a file using pickle
with open('rfr_model_1_2.pkl', 'wb') as file:
    pickle.dump(best_model, file)


# Actual vs Predicted Plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_test, c='blue', marker='o', label='Test data')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linewidth=2)
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('Actual vs Predicted Plot')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()


### Model 1.3 (top n features)

In [ ]:

# Extract features and target
X = df[lagged_tr_features + ['been fatigued (no energy/very tired)_frequency_lag1', 'been fatigued (no energy/very tired)_onset_lag1', 
                            'cold or flu symptoms_onset_lag1', 'been dizzy or unsteady on your feet_frequency_lag1']]
y = df['TR']

# Separate the binary and numeric features
numeric_features = lagged_tr_features 

# Scale numeric features using QuantileTransformer
scaler = QuantileTransformer(output_distribution='normal')
X[numeric_features] = scaler.fit_transform(X[numeric_features])

# Train-test split
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

# Hyperparameter grid for RandomForestRegressor
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV for hyperparameter tuning
model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print(f"Best hyperparameters:\n{best_params}")

# Evaluate on training set
y_pred_train = best_model.predict(X_train)
train_mae = mean_absolute_error(y_train, y_pred_train)
train_r2 = r2_score(y_train, y_pred_train)
print(f"Train MAE: {train_mae}")
print(f"Train R-squared: {train_r2}")

# Evaluate on validation set
y_pred_val = best_model.predict(X_val)
val_mae = mean_absolute_error(y_val, y_pred_val)
val_r2 = r2_score(y_val, y_pred_val)
print(f"Validation MAE: {val_mae}")
print(f"Validation R-squared: {val_r2}")

# Evaluate on test set
y_pred_test = best_model.predict(X_test)
test_mae = mean_absolute_error(y_test, y_pred_test)
test_r2 = r2_score(y_test, y_pred_test)
print(f"Test MAE: {test_mae}")
print(f"Test R-squared: {test_r2}")

# Print best hyperparameters
print(f"Best hyperparameters:\n{best_params}")

# Plot feature importances for the best model
feature_importances = best_model.feature_importances_
features = X_train.columns

indices = np.argsort(feature_importances)[::-1]

plt.figure(figsize=(10, 6))
plt.title("Feature Importances")
plt.bar(range(X_train.shape[1]), feature_importances[indices], align="center")
plt.xticks(range(X_train.shape[1]), features[indices], rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()
plt.show()

# Save the model to a file using pickle
with open('rfr_model_1_3.pkl', 'wb') as file:
    pickle.dump(best_model, file)


# Actual vs Predicted Plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_test, c='blue', marker='o', label='Test data')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linewidth=2)
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('Actual vs Predicted Plot')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()


## Predictive modeling for Symptom reports

### Model 2.1 (symptom-based)

In [ ]:
X = df[lagged_symptom_features + lagged_tr_features + ['TR'] + lagged_temporal_features]
y = df[prob_b_columns]

# Separate the binary and numeric features
numeric_features = lagged_tr_features + ['TR']

# Scale numeric features using QuantileTransformer
scaler = QuantileTransformer(output_distribution='normal')
X[numeric_features] = scaler.fit_transform(X[numeric_features])

# Check shapes of X and y to ensure they are consistent
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

# Ensure the indices of X and y match
X, y = X.align(y, join='inner', axis=0)
print("Shape of X after alignment:", X.shape)
print("Shape of y after alignment:", y.shape)

# Train-test split
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

# Hyperparameter grid for RandomForestClassifier
param_grid = {
    'estimator__n_estimators': [50, 100, 200],
    'estimator__max_depth': [10, 20, None],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 4]
}

# GridSearchCV for hyperparameter tuning
base_model = RandomForestClassifier(random_state=42)
model = MultiOutputClassifier(base_model)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print(f"Best hyperparameters:\n{best_params}")

# Evaluate on training set
y_pred_train = best_model.predict(X_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')
print(f"Train F1: {train_f1}")

# Evaluate on validation set
y_pred_val = best_model.predict(X_val)
val_f1 = f1_score(y_val, y_pred_val, average='macro')
print(f"Validation F1: {val_f1}")

# Evaluate on test set
y_pred_test = best_model.predict(X_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')
print(f"Test F1: {test_f1}")

# Calculate PR AUC for test set
precision_list = []
recall_list = []
pr_auc_list = []

y_pred_prob_test = np.stack([best_model.estimators_[i].predict_proba(X_test)[:, 1] for i in range(len(prob_b_columns))], axis=1)
for i, col in enumerate(prob_b_columns):
    precision, recall, _ = precision_recall_curve(y_test.iloc[:, i], y_pred_prob_test[:, i])
    pr_auc = auc(recall, precision)
    precision_list.append(precision)
    recall_list.append(recall)
    pr_auc_list.append(pr_auc)
    plt.plot(recall, precision, lw=2, label=f'{col} (AUC = {pr_auc:.2f})')

# Print overall scores
print("Average Train F1:", train_f1)
print("Average Validation F1:", val_f1)
print("Average Test F1:", test_f1)

# Print best hyperparameters
print(f"Best hyperparameters:\n{best_params}")

# Save the model to a file using pickle
with open('rfc_model_2_1.pkl', 'wb') as file:
    pickle.dump(best_model, file)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('PR Curve for test set')
plt.legend(loc='best')
plt.tight_layout()
plt.show()


### Model 2.2 (symptom-disease)

In [ ]:
X = df[lagged_symptom_features + lagged_tr_features + ['TR'] + lagged_temporal_features + disease_columns + lagged_outcomes]
y = df[prob_b_columns]

# Separate the binary and numeric features
numeric_features = lagged_tr_features + ['TR']

# Scale numeric features using QuantileTransformer
scaler = QuantileTransformer(output_distribution='normal')
X[numeric_features] = scaler.fit_transform(X[numeric_features])

# Check shapes of X and y to ensure they are consistent
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

# Ensure the indices of X and y match
X, y = X.align(y, join='inner', axis=0)
print("Shape of X after alignment:", X.shape)
print("Shape of y after alignment:", y.shape)

# Train-test split
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

# Hyperparameter grid for RandomForestClassifier
param_grid = {
    'estimator__n_estimators': [50, 100, 200],
    'estimator__max_depth': [10, 20, None],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 4]
}

# GridSearchCV for hyperparameter tuning
base_model = RandomForestClassifier(random_state=42)
model = MultiOutputClassifier(base_model)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print(f"Best hyperparameters:\n{best_params}")

# Evaluate on training set
y_pred_train = best_model.predict(X_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')
print(f"Train F1: {train_f1}")

# Evaluate on validation set
y_pred_val = best_model.predict(X_val)
val_f1 = f1_score(y_val, y_pred_val, average='macro')
print(f"Validation F1: {val_f1}")

# Evaluate on test set
y_pred_test = best_model.predict(X_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')
print(f"Test F1: {test_f1}")

# Calculate PR AUC for test set
precision_list = []
recall_list = []
pr_auc_list = []

y_pred_prob_test = np.stack([best_model.estimators_[i].predict_proba(X_test)[:, 1] for i in range(len(prob_b_columns))], axis=1)
for i, col in enumerate(prob_b_columns):
    precision, recall, _ = precision_recall_curve(y_test.iloc[:, i], y_pred_prob_test[:, i])
    pr_auc = auc(recall, precision)
    precision_list.append(precision)
    recall_list.append(recall)
    pr_auc_list.append(pr_auc)
    plt.plot(recall, precision, lw=2, label=f'{col} (AUC = {pr_auc:.2f})')

# Print overall scores
print("Average Train F1:", train_f1)
print("Average Validation F1:", val_f1)
print("Average Test F1:", test_f1)

# Print best hyperparameters
print(f"Best hyperparameters:\n{best_params}")

# Save the model to a file using pickle
with open('rfc_model_2_2.pkl', 'wb') as file:
    pickle.dump(best_model, file)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('PR Curve for test set')
plt.legend(loc='best')
plt.tight_layout()
plt.show()


### Model 2.3

In [ ]:
X = df[['pain or stiffness in your joints_duration_lag1', 'pain or stiffness in your joints_onset_lag1', 
        'pain or stiffness in your joints_frequency_lag1', 'pain or stiffness in your back_frequency_lag1',
        'been fatigued (no energy/very tired)_frequency_lag1']]
y = df[prob_b_columns]

# Separate the binary and numeric features
#numeric_features = lagged_tr_features + ['TR']

# Scale numeric features using QuantileTransformer
#scaler = QuantileTransformer(output_distribution='normal')
#X[numeric_features] = scaler.fit_transform(X[numeric_features])

# Check shapes of X and y to ensure they are consistent
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

# Ensure the indices of X and y match
X, y = X.align(y, join='inner', axis=0)
print("Shape of X after alignment:", X.shape)
print("Shape of y after alignment:", y.shape)

# Train-test split
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

# Hyperparameter grid for RandomForestClassifier
param_grid = {
    'estimator__n_estimators': [50, 100, 200],
    'estimator__max_depth': [10, 20, None],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 4]
}

# GridSearchCV for hyperparameter tuning
base_model = RandomForestClassifier(random_state=42)
model = MultiOutputClassifier(base_model)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print(f"Best hyperparameters:\n{best_params}")

# Evaluate on training set
y_pred_train = best_model.predict(X_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')
print(f"Train F1: {train_f1}")

# Evaluate on validation set
y_pred_val = best_model.predict(X_val)
val_f1 = f1_score(y_val, y_pred_val, average='macro')
print(f"Validation F1: {val_f1}")

# Evaluate on test set
y_pred_test = best_model.predict(X_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')
print(f"Test F1: {test_f1}")

# Calculate PR AUC for test set
precision_list = []
recall_list = []
pr_auc_list = []

y_pred_prob_test = np.stack([best_model.estimators_[i].predict_proba(X_test)[:, 1] for i in range(len(prob_b_columns))], axis=1)
for i, col in enumerate(prob_b_columns):
    precision, recall, _ = precision_recall_curve(y_test.iloc[:, i], y_pred_prob_test[:, i])
    pr_auc = auc(recall, precision)
    precision_list.append(precision)
    recall_list.append(recall)
    pr_auc_list.append(pr_auc)
    plt.plot(recall, precision, lw=2, label=f'{col} (AUC = {pr_auc:.2f})')

# Print overall scores
print("Average Train F1:", train_f1)
print("Average Validation F1:", val_f1)
print("Average Test F1:", test_f1)

# Print best hyperparameters
print(f"Best hyperparameters:\n{best_params}")

# Save the model to a file using pickle
with open('rfc_model_2_3.pkl', 'wb') as file:
    pickle.dump(best_model, file)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('PR Curve for test set')
plt.legend(loc='best')
plt.tight_layout()
plt.show()


## Predictive modeling for TUC

In [ ]:
df['event_observed'] = df['combined_change'].astype(bool)

In [ ]:
# Determine the number of people who never have event_observed
never_event_observed = df.groupby('studyID')['event_observed'].sum() == 0
num_never_event_observed = never_event_observed.sum()
print(f"Number of people who never have event_observed: {num_never_event_observed}")

### Model 3.1 (symptom-based)

In [ ]:

# Extract features and target
X = df[prob_b_columns + lagged_tr_features + lagged_temporal_features]
y = df['actual_time_until_change']

# Separate the binary and numeric features
numeric_features = lagged_tr_features

# Scale numeric features using QuantileTransformer
scaler = QuantileTransformer(output_distribution='normal')
X[numeric_features] = scaler.fit_transform(X[numeric_features])

# Manually split the data into training and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

# Hyperparameter grid for RandomForestRegressor
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV for hyperparameter tuning on the training set with 5-fold cross-validation
model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print(f"Best hyperparameters:\n{best_params}")

# Evaluate on training set
y_pred_train = best_model.predict(X_train)
train_mae = mean_absolute_error(y_train, y_pred_train)
train_r2 = r2_score(y_train, y_pred_train)
print(f"Train MAE: {train_mae}")
print(f"Train R-squared: {train_r2}")

# Evaluate on validation set
y_pred_val = best_model.predict(X_val)
val_mae = mean_absolute_error(y_val, y_pred_val)
val_r2 = r2_score(y_val, y_pred_val)
print(f"Validation MAE: {val_mae}")
print(f"Validation R-squared: {val_r2}")

# Evaluate on test set
y_pred_test = best_model.predict(X_test)
test_mae = mean_absolute_error(y_test, y_pred_test)
test_r2 = r2_score(y_test, y_pred_test)
print(f"Test MAE: {test_mae}")
print(f"Test R-squared: {test_r2}")


# Save the model to a file using pickle
with open('rfr_model_3_1.pkl', 'wb') as file:
    pickle.dump(best_model, file)


# Actual vs Predicted Plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_test, c='blue', marker='o', label='Test data')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linewidth=2)
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('Actual vs Predicted Plot')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()


### Model 3.2 (symptom-disease-outcome)

In [ ]:

# Extract features and target
X = df[prob_b_columns + lagged_tr_features + lagged_temporal_features + disease_columns + lagged_outcomes]
y = df['actual_time_until_change']

# Separate the binary and numeric features
numeric_features = lagged_tr_features

# Scale numeric features using QuantileTransformer
scaler = QuantileTransformer(output_distribution='normal')
X[numeric_features] = scaler.fit_transform(X[numeric_features])

# Manually split the data into training and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

# Hyperparameter grid for RandomForestRegressor
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV for hyperparameter tuning on the training set with 5-fold cross-validation
model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print(f"Best hyperparameters:\n{best_params}")

# Evaluate on training set
y_pred_train = best_model.predict(X_train)
train_mae = mean_absolute_error(y_train, y_pred_train)
train_r2 = r2_score(y_train, y_pred_train)
print(f"Train MAE: {train_mae}")
print(f"Train R-squared: {train_r2}")

# Evaluate on validation set
y_pred_val = best_model.predict(X_val)
val_mae = mean_absolute_error(y_val, y_pred_val)
val_r2 = r2_score(y_val, y_pred_val)
print(f"Validation MAE: {val_mae}")
print(f"Validation R-squared: {val_r2}")

# Evaluate on test set
y_pred_test = best_model.predict(X_test)
test_mae = mean_absolute_error(y_test, y_pred_test)
test_r2 = r2_score(y_test, y_pred_test)
print(f"Test MAE: {test_mae}")
print(f"Test R-squared: {test_r2}")


# Save the model to a file using pickle
with open('rfr_model_3_2.pkl', 'wb') as file:
    pickle.dump(best_model, file)


# Actual vs Predicted Plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_test, c='blue', marker='o', label='Test data')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linewidth=2)
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('Actual vs Predicted Plot')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()


### Model 3.3 (top n features)

In [ ]:

# Extract features and target
X = df[lagged_tr_features + ['been fatigued (no energy/very tired)_onset_lag1', 'been fatigued (no energy/very tired)_frequency_lag1', 
                            'nausea, vomiting, diarrhea, or other stomach (abdominal) problem_frequency_lag1', 
                             'pain or stiffness in your joints_frequency_lag1']]
y = df['actual_time_until_change']

# Separate the binary and numeric features
numeric_features = lagged_tr_features

# Scale numeric features using QuantileTransformer
scaler = QuantileTransformer(output_distribution='normal')
X[numeric_features] = scaler.fit_transform(X[numeric_features])

# Manually split the data into training and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

# Hyperparameter grid for RandomForestRegressor
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV for hyperparameter tuning on the training set with 5-fold cross-validation
model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print(f"Best hyperparameters:\n{best_params}")

# Evaluate on training set
y_pred_train = best_model.predict(X_train)
train_mae = mean_absolute_error(y_train, y_pred_train)
train_r2 = r2_score(y_train, y_pred_train)
print(f"Train MAE: {train_mae}")
print(f"Train R-squared: {train_r2}")

# Evaluate on validation set
y_pred_val = best_model.predict(X_val)
val_mae = mean_absolute_error(y_val, y_pred_val)
val_r2 = r2_score(y_val, y_pred_val)
print(f"Validation MAE: {val_mae}")
print(f"Validation R-squared: {val_r2}")

# Evaluate on test set
y_pred_test = best_model.predict(X_test)
test_mae = mean_absolute_error(y_test, y_pred_test)
test_r2 = r2_score(y_test, y_pred_test)
print(f"Test MAE: {test_mae}")
print(f"Test R-squared: {test_r2}")


# Save the model to a file using pickle
with open('rfr_model_3_3.pkl', 'wb') as file:
    pickle.dump(best_model, file)


# Actual vs Predicted Plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_test, c='blue', marker='o', label='Test data')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linewidth=2)
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('Actual vs Predicted Plot')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()


##### Author: Nikhil Kashyap, MSc Information Studies: Data Science, UvA